## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
Sparse graph recovery by optimizing deep unrolled networks. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
    - Novel `consensus` strategy which robustly handles missing values by leveraging the multi-task learning ability of the model   
    - Multi-task learning mode that solves the graphical lasso objective to recover multiple graphs with a single `uGLAD` model  
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/uGLAD


In [2]:
import torch
torch.__version__

'1.10.1'

# 1. Synthetic data convergence

In [25]:
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=10, 
    sparsity=[0.2, 0.2], 
    num_samples=500, 
    batch_size=1,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 10, 10), Samples (1, 500, 10)


### The uGLAD model

Learning details:  
1. Initialize learnable `GLAD` parameters  
2. Run the GLAD model  
3. Get the glasso-loss  
4. Backprop  

Possible solutions if `uGLAD` does not converge:  
1. Increase number of training EPOCHS
2. Lower the learning rate    
3. Please re-run. This will run the optimization with different initializations  
4. Change the INIT_DIAG=0/1 in the `GLAD` model parameters  
5. Increase `L`, the number of unrolled iterations of `GLAD`

### Running the uGLAD-Direct mode

- Directly optimize the uGLAD model on the complete data X
- Optimizes the model to minimize the glasso-loss on X 

In [26]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=300,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False, 
    k_fold=0,  # Direct mode
    mode='direct'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Direct Mode
Total runtime: 17.319546699523926 secs

uGLAD: {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 12, 'nnzPred': 12, 'precision': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}


### Running the uGLAD-CV mode 

- Finds the best model by doing cross-fold validation on the input samples X
- Chooses the model which performs best in terms of glasso-loss on held-out data
- More conservative than the direct mode

In [5]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=250,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=3, 
    mode='cv'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
CV mode: 3-fold
Total runtime: 37.335291147232056 secs

uGLAD: {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 10, 'nnzPred': 10, 'precision': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}


### Comparison with sklearn's GraphicalLassoCV

In [6]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 0.6551724137931034, 'TPR': 1.0, 'FPR': 0.5428571428571428, 'SHD': 19, 'nnzTrue': 10, 'nnzPred': 29, 'precision': 0.3448275862068966, 'recall': 1.0, 'Fbeta': 0.5128205128205128, 'aupr': 1.0, 'auc': 1.0}


# 2. Handling missing values
Running `uGLAD` model in mode=`missing`:
- Leverages the multi-task learning feature of the `uGLAD` model
- Uses the novel `consensus` strategy to robustly handle the missing values

In [14]:
# Adding dropout noise to Xb
from uGLAD.utils.prepare_data import add_noise_dropout
from uGLAD.main import mean_imputation
import numpy as np

# Adding np.NaNs to introduce missing values
Xb_miss = add_noise_dropout(Xb, dropout=0.90)
# Doing mean imputation for basic statistical comparsion
B, M, D = Xb_miss.shape
Xb_mean = [] 
for b in range(B):
    X_miss = Xb_miss[b].copy()
    X_miss = X_miss.reshape(1, M, D)
    Xb_mean.append(mean_imputation(X_miss).reshape(M, D))
Xb_mean = np.array(Xb_mean)

### Running the `uGLAD-miss` model in missing data mode 

In [17]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb_miss[0],
    centered=False,
    epochs=500,
    lr=0.01,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=3,  # The number of sumsample splits
    mode='missing'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Handling missing data
Creating K=3 row-subsampled batches
Getting the final precision matrix using the consensus strategy
Total runtime: 40.34000372886658 secs

uGLAD: {'FDR': 0.4, 'TPR': 0.3, 'FPR': 0.05714285714285714, 'SHD': 9, 'nnzTrue': 10, 'nnzPred': 5, 'precision': 0.6, 'recall': 0.3, 'Fbeta': 0.4, 'aupr': 0.3655555555555555, 'auc': 0.6185714285714285}


### Comparison with BCD-mean
Run GraphicalLassoCV with mean imputed Xb_mean

In [16]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb_mean[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 1.0, 'TPR': 0.0, 'FPR': 0.02857142857142857, 'SHD': 11, 'nnzTrue': 10, 'nnzPred': 1, 'precision': 0.0, 'recall': 0.0, 'Fbeta': 0.0, 'aupr': 0.2222222222222222, 'auc': 0.4857142857142857}


# 3. Multi-task learning mode
- Generate synthetic data coming from graphs with varying sparsity
- Recover the batch precision matrices for the batch input data X

In [30]:
# Creating synthetic data for multi-task learning 
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=10, 
    sparsity=[0.1, 0.2], 
    num_samples=500, 
    batch_size=3,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (3, 10, 10), Samples (3, 500, 10)


In [46]:
# Running uGLAD in multi-task learning mode
from uGLAD import main as uG
from uGLAD.utils.metrics import summarize_compare_theta

# Initialize the model
model_uGLAD = uG.uGLAD_multitask()  

K, M, D = Xb.shape

# Fit to the data
model_uGLAD.fit(
    Xb,
    centered=False,
    epochs=200,
    lr=0.01,
    INIT_DIAG=0,
    L=15,
    verbose=False,
)

# Print the compare metrics
compare_theta_MT = []
for b in range(K):
    rM = reportMetrics(
            true_theta[b], 
            model_uGLAD.precision_[b]
        )
    print(f'Metrics for graph {b}: {rM}\n')
    compare_theta_MT.append(rM)

# Calculate the average statistics
summarize_compare_theta(compare_theta_MT, method_name='uGLAD multi-task')

Running uGLAD in multi-task mode
Total runtime: 17.185999155044556 secs

Metrics for graph 0: {'FDR': 0.8205128205128205, 'TPR': 1.0, 'FPR': 0.8421052631578947, 'SHD': 32, 'nnzTrue': 7, 'nnzPred': 39, 'precision': 0.1794871794871795, 'recall': 1.0, 'Fbeta': 0.30434782608695654, 'aupr': 0.9999999999999998, 'auc': 1.0}

Metrics for graph 1: {'FDR': 0.7777777777777778, 'TPR': 1.0, 'FPR': 0.7567567567567568, 'SHD': 28, 'nnzTrue': 8, 'nnzPred': 36, 'precision': 0.2222222222222222, 'recall': 1.0, 'Fbeta': 0.36363636363636365, 'aupr': 1.0, 'auc': 1.0}

Metrics for graph 2: {'FDR': 0.7941176470588235, 'TPR': 1.0, 'FPR': 0.7105263157894737, 'SHD': 27, 'nnzTrue': 7, 'nnzPred': 34, 'precision': 0.20588235294117646, 'recall': 1.0, 'Fbeta': 0.34146341463414637, 'aupr': 0.9999999999999998, 'auc': 1.0}

Avg results for uGLAD multi-task

{'FDR': (0.797469415116474, 0.017606754976915266),
 'FPR': (0.7697961119013751, 0.05450243685430367),
 'Fbeta': (0.3364825347858222, 0.024459347827697635),
 'SHD': (2